## Table Question-Answering 모델을 이용해서 파이썬 에러 질의응답 서비스 만들어보기
---
허깅페이스(Huggingface)의 트랜스포머(Transformers) 패키지는 파이썬(Python)을 이용한 자연어 처리(Natural Language Processing, NLP) 작업을 아주 간단하게 만들어 주었습니다. 특히, 여러가지 모델을 이용하고 싶을 때 똑같은 파이프라인에 모델 이름만 바꿔넣으면서 사용할 수 있어서 파이썬 기초만 뗀 사람이 바로 NLP 모델을 이용한 서비스를 만들 수도 있게 되었지요.
</br></br>

그래서, 이번에는 NLP를 이용한 간단한 서비스를 하나 만들어 보도록 하겠습니다. 자연어 처리 중 Table Question-Answering 작업에 해당하는 서비스입니다. 사전 지식을 표 형태로 입력해 주면, 자연어로 질문을 했을 때 자연어 모델이 제법 정확하게 답변을 해 주는 방식입니다.
</br></br>

자연어 처리 모델, 그중에서 우리가 챗봇 형태의 모델로 잘 알고있는 대화형(Conversational) 모델의 가장 큰 단점은 해당 모델이 참/거짓을 판단하지 않고 대답한다는 사실입니다. 대표적으로, 챗GPT(ChatGPT)를 이용해서 다양한 질문을 해 보면, 결코 낮지 않은 비율로 가짜 정보를 그럴듯하게 대답하는 상황이 벌어집니다.
</br></br>

이 때문에, 실제 자연어 처리 모델을 이용한 챗봇은 전문적인 서비스에서는 조심스럽게 사용을 검토하고 있는 상황인데요, 이 때 과도기적 기술로 사용할 수 있는게 바로 표 질의응답(Table Question-Answering) 모델입니다.
</br></br>

표 질의응답 모델은, 자연어를 인식하고 처리하는 과정은 크게 다르지 않지만, 답변할 수 있는 내용이 주어진 표로 한정된다는 점에서 차이를 보입니다. 만약 질문이 제한된 범위에서만 들어올 경우에는 표를 잘 만들어 두기만 하면 예상치 못한 답변을 걱정하지 않아도 된다는 점이죠. 실제로 AI 고객문의 서비스로 종종 사용되고 있는 모델입니다.
</br></br>

표 질의응답 모델과 기존 검색엔진의 차이점은, 질문하는 사람이 키워드를 신경쓰지 않고 자연스럽게 물어봐도 된다는 점입니다. 전통적인 검색엔진은 주어진 DB에서 일치하는 단어 또는 문장을 찾아내는 방식인데, 웹사이트 검색을 자주 하는 분들이라면 똑같은 표현이라도 키워드에 따라서 다른 검색 결과가 나오는걸 확인할 수 있을거예요. **검색하는 것도 능력이다**라는 말이 있는 이유입니다.
</br></br>

하지만, 표 질의응답 모델의 경우에는 굳이 키워드 일치를 확인하지 않고 자연어 모델이 문맥을 파악하는 방식입니다. 기술적으로 들어가면 **파악**이라는 말이 적확한 표현은 아니지만, 개념상으로는 그렇게 동작한다고 이해하시면 편할겁니다.
</br></br>

이러한 특징을 이용해서 저는 파이썬 코딩을 하다가 문제가 생겼을 때, 에러에 대한 정보를 얻을 수 있도록 질의응답 서비스를 만들어보려 합니다. 지금은 파이썬 공식 문서의 예외 항목을 참고해서 표를 만들었는데, 만약 이 데이터를 조금 더 늘린다면 폭넓은 답변을 할 수 있을겁니다.
</br></br>
---


In [1]:
from transformers import pipeline
import pandas as pd

</br></br>

Huggingface Transformers는 pipeline이라는 모듈을 제공하고 있습니다. 이 파이프라인이라는 모듈은 말 그대로 작업(Task)에 따라서 모델 이름만 입력해주면 규격화된 파이프라인을 사용할 수 있도록 해 주는데요, 저는 앞서 알려드렸던 Table Question Answering 작업을 사용해 보겠습니다. pipeline에는 ```task='table-question-answering'``` 이렇게 입력을 해 주면 됩니다.
</br></br>

다음으로는 모델인데요, 저는 **koreapas-finetuned-korwikitq** 모델(https://huggingface.co/dsba-lab/koreapas-finetuned-korwikitq)을 활용해 보려 합니다. 많은 분들이 대화형 모델에 집중하고 있어서 적절한 한국어 표 질의응답 모델 선택의 폭이 넓지는 않은데, 그중에서 이번 서비스 개발에 딱 들어맞는 괜찮은 모델입니다.
</br></br>


In [2]:
qa_model = pipeline(task='table-question-answering',
                  model='dsba-lab/koreapas-finetuned-korwikitq')

# 답변용 데이터 불러오기
df = pd.read_csv('./python_exception.csv')

</br></br>

여기서 답변용 데이터의 형식을 살펴봅시다. 표 질의응답 모델에서는 답변용 데이터의 형식이 많이 중요한데요, 저는 판다스 데이터프레임(Pandas DataFrame) 형식으로 입력하기 위해 csv 데이터를 작성했습니다.
</br></br>

내용은 파이썬에서 흔히 발생하는 에러의 이름과, 그에 대한 설명을 입력했습니다. 개인적으로 테스트를 해 보니, 최대한 혼동되는 어휘가 없게끔 집어넣어야 그럴듯한 대답을 해 줍니다. 그리고 설명은 줄글로 막 써도 되는데요, 특정 요소에 따라서 데이터를 분리해 두지 않아도 된다는 점은 표 질의응답 모델의 매력입니다.
</br></br>


In [3]:
def make_answer(q):
    answer = qa_model(query=q, table=df)

    if answer['answer'] == '':
        return '질문에 대한 답을 찾지 못했습니다.'

    elif answer['coordinates'][0][-1] == 0:
        answer2 = answer['answer']

        return f'''질문이나 코드에서 파악한 에러의 이름은 {answer2} 입니다.
    에러가 발생하는 원인은 다음과 같습니다: {df[df['Error'] == answer2]['설명'].iloc[0]}'''

    else:
        answer2 = answer['cells'][0]

        return f'''질문이나 코드에서 파악한 문제는 {answer2}
이 에러의 이름은 {df[df['설명'] == answer2]['Error'].iloc[0]} 입니다.'''

</br></br>

다음으로는, 미리 불러온 모델에 질문을 입력해주고, 답변들이 적힌 표를 함께 입력해 주는 과정으로 끝납니다. 질문은 ```query```, 답변은 ```table```로 표현합니다.
</br></br>

하지만, 이렇게만 사용한다면 표에서 검색 결과를 그대로 적어주기만 하는 다소 딱딱한 모델이 되어버리는데요, 이를 방지하기 위해서는 두 가지 고민을 해볼 수 있습니다.
</br></br>

1. 규격화된 답변에 맞춰서 출력문을 손봐주거나,
1. Tezt2Text Generation 모델을 이용해서 NLP 모델이 스스로 문장을 구성하게 해 주는것 말이죠.
</br></br>

여기서 두 번째를 고려하려면 고려할 사항이 많아지니까, 일단은 첫 번째 선택지를 골라봅시다. 허깅페이스 트랜스포머에서 표 질의응답 모델은 응답을 딕셔너리(Dictionary)로 반환하는데, 응답을 ```answer```, 해당 셀의 위치를 ```coordinates```, 그 셀의 내용을 ```cells```로 표현합니다.
</br></br>

만약 적절한 응답이 없다면 ```answer```가 빈 문자열일테니 이를 인식해 답변이 없다고 반환해주고, 에러 이름과 설명 중 무엇을 찾았는지를 기준으로 해서, 자연스러운 응답을 구성해 적어줍니다.
</br></br>


In [4]:
df.head()

,Error,설명
0,AssertionError,assert 문이 실패할 때 발생합니다.
1,AttributeError,어트리뷰트 참조(어트리뷰트 참조를 보세요)나 대입이 실패할 때 발생합니다. (객체가...
2,EOFError,input() 함수가 데이터를 읽지 못한 상태에서 EOF (end-of-file) ...
3,FloatingPointError,현재 사용되지 않습니다.
4,GeneratorExit,제너레이터 또는 코루틴 이 닫힐 때 발생합니다; generator.close() 와...


In [5]:
# 질문 입력
inputs = input('질문이나 에러 메시지를 입력해 주세요: ')

질문이나 에러 메시지를 입력해 주세요:  질문이나 에러 메시지를 입력해 주세요:  12/0으로 나누기 하는 코드를 실행했는데 에러가 나잖아. 왜 이러는거야?


In [6]:
return_msg = make_answer(inputs)

In [7]:
print(return_msg)

질문이나 코드에서 파악한 에러의 이름은 ZeroDivisionError 입니다.
    에러가 발생하는 원인은 다음과 같습니다: 나누기 또는 모듈로 연산의 두 번째 인자가 0일 때 발생합니다. 연관된 값은 피연산자의 형과 연산을 나타내는 문자열입니다.


</br></br>

---

이렇게 허깅페이스 트랜스포머 패키지의 파이프라인을 이용해서, Table Question-Answering 모델이 적용된 파이썬 에러 질의응답 서비스를 만들어 봤습니다. 관련 지식을 모르고도 손쉽게 NLP 프로그램을 구현할 수 있기 때문에 만약 이런 서비스에 관심이 있다면 따라만 해 보시면 좋겠습니다.
</br></br>
물론, 응답이 너무 제한적이라거나 기계적인 인상이 있지만, 이런 문제는 다음에 차차 개선해 보도록 하겠습니다. 만약에 코드가 필요하다면, 화면을 드래그해서 복사해 보시거나, 깃허브에서 ipynb 파일을 다운로드해 보시기 바랍니다.
</br></br>
